# SNARS – Community Detection Competition (OWN Implementation, Efficient, Robust)

This version fixes a critical K-means convergence bug (previously `prev_inertia=inf` caused an immediate break and `best_labels=None`).

Implemented by us:
- Robust adjacency CSV reader (auto-sep, NaN cleaning)
- Normalized Laplacian
- Eigengap K estimation (UNC)
- **K-means from scratch** (k-means++ + vectorized distances + multiple restarts)
- SNARS output formatting


In [1]:
# ==============================
# CONFIGURATION (EDIT THESE)
# ==============================

GROUP_NAME = "Muhammad"
AUTHORS = "Muhammad Fahim Asim"
REPO_URL = "https://github.com/muhammad-fahim-asim/Community-Detection---Social-Networks-Recommendation-Systems"

BENCHMARK_DIR = "Dataset"   # where the benchmark CSVs are
OUTPUT_ROOT = "."     # write outputs next to this notebook

USE_SCIPY_EIGSH = True
K_MAX_UNC = 30
RANDOM_SEED = 0


In [2]:
import os, glob, re, time
import numpy as np
import pandas as pd

np.random.seed(RANDOM_SEED)

_HAVE_SCIPY = False
if USE_SCIPY_EIGSH:
    try:
        import scipy.sparse as sp
        from scipy.sparse.linalg import eigsh
        _HAVE_SCIPY = True
    except Exception:
        _HAVE_SCIPY = False
        print("SciPy not available; using NumPy eigen decomposition.")

In [3]:
def list_benchmark_files(benchmark_dir: str):
    return sorted(glob.glob(os.path.join(benchmark_dir, "*.csv")))

def parse_k_from_filename(path: str):
    m = re.search(r"K\s*=\s*(\d+)", os.path.basename(path))
    return int(m.group(1)) if m else None

def read_adjacency_csv(path: str) -> np.ndarray:
    df = pd.read_csv(path, header=None, sep=None, engine="python")
    df = df.apply(pd.to_numeric, errors="coerce")
    df = df.dropna(axis=0, how="all").dropna(axis=1, how="all")
    df = df.fillna(0.0)
    A = df.to_numpy(dtype=float)

    if A.shape[0] != A.shape[1]:
        raise ValueError(f"Adjacency matrix must be square. Got {A.shape} for {path}")

    if not np.isfinite(A).all():
        bad = int(np.size(A) - np.isfinite(A).sum())
        raise ValueError(f"Adjacency has {bad} non-finite values after cleaning: {path}")

    np.fill_diagonal(A, 0.0)
    A = 0.5 * (A + A.T)
    return A

def normalized_laplacian(A: np.ndarray):
    n = A.shape[0]
    deg = A.sum(axis=1)
    inv_sqrt = np.zeros_like(deg)
    mask = deg > 1e-12
    inv_sqrt[mask] = 1.0 / np.sqrt(deg[mask])
    S = (A * inv_sqrt[:, None]) * inv_sqrt[None, :]
    L = np.eye(n) - S
    return L, deg

In [4]:
# ------------------------------
# OWN K-MEANS (fixed + efficient)
# ------------------------------

def _kmeans_plus_plus_init(X: np.ndarray, k: int, rng: np.random.Generator):
    n, d = X.shape
    centers = np.empty((k, d), dtype=float)
    centers[0] = X[rng.integers(0, n)]
    dist2 = np.sum((X - centers[0])**2, axis=1)

    for j in range(1, k):
        total = dist2.sum()
        if total <= 1e-12:
            centers[j:] = X[rng.integers(0, n, size=(k-j))]
            break
        probs = dist2 / total
        centers[j] = X[rng.choice(n, p=probs)]
        new_dist2 = np.sum((X - centers[j])**2, axis=1)
        dist2 = np.minimum(dist2, new_dist2)
    return centers

def kmeans_own(
    X: np.ndarray,
    k: int,
    n_init: int = 8,
    max_iter: int = 120,
    tol: float = 1e-5,
    random_state: int = 0
):
    if not np.isfinite(X).all():
        raise ValueError("kmeans_own received NaN/inf in X.")

    n, d = X.shape
    rng = np.random.default_rng(random_state)

    best_inertia = np.inf
    best_labels = None

    X_norm2 = np.sum(X * X, axis=1, keepdims=True)  # (n,1)

    for run in range(n_init):
        centers = _kmeans_plus_plus_init(X, k, rng)
        centers_norm2 = np.sum(centers * centers, axis=1, keepdims=True).T  # (1,k)

        prev_inertia = None  # IMPORTANT: avoid inf bug
        for it in range(max_iter):
            dist2 = X_norm2 + centers_norm2 - 2.0 * (X @ centers.T)  # (n,k)
            labels = np.argmin(dist2, axis=1)

            # update centers
            new_centers = np.zeros_like(centers)
            counts = np.bincount(labels, minlength=k).astype(float)
            for j in range(k):
                if counts[j] > 0:
                    new_centers[j] = X[labels == j].mean(axis=0)
                else:
                    new_centers[j] = X[rng.integers(0, n)]
            centers = new_centers
            centers_norm2 = np.sum(centers * centers, axis=1, keepdims=True).T

            inertia = float(np.sum(np.min(dist2, axis=1)))
            if not np.isfinite(inertia):
                prev_inertia = None
                break

            if prev_inertia is not None:
                if abs(prev_inertia - inertia) <= tol * max(1.0, prev_inertia):
                    break
            prev_inertia = inertia

        if prev_inertia is not None and prev_inertia < best_inertia:
            best_inertia = prev_inertia
            best_labels = labels.copy()

    if best_labels is None:
        raise ValueError("kmeans_own failed: best_labels is None (unexpected).")

    return best_labels, best_inertia

In [5]:
def row_normalize(U: np.ndarray, eps: float = 1e-12):
    norms = np.linalg.norm(U, axis=1, keepdims=True)
    return U / np.maximum(norms, eps)

def smallest_eigs(L: np.ndarray, k: int):
    n = L.shape[0]
    k = int(min(k, n-1))
    if _HAVE_SCIPY and n >= 250:
        vals, vecs = eigsh(sp.csr_matrix(L), k=k, which="SM")
        order = np.argsort(vals)
        return np.real(vals[order]), np.real(vecs[:, order])
    vals, vecs = np.linalg.eigh(L)
    return np.real(vals[:k]), np.real(vecs[:, :k])

def estimate_k_eigengap(L: np.ndarray, k_max: int = 30):
    n = L.shape[0]
    k_max = int(min(k_max, max(2, n-1)))
    vals, _ = smallest_eigs(L, k_max)
    gaps = np.diff(vals)
    if len(gaps) < 2:
        return 2, vals
    start = 1
    idx = int(np.argmax(gaps[start:]) + start)
    return max(2, idx + 1), vals

def spectral_clustering_own(A: np.ndarray, k: int, random_state: int = 0):
    L, _ = normalized_laplacian(A)
    _, vecs = smallest_eigs(L, k)
    U = row_normalize(vecs)
    if not np.isfinite(U).all():
        raise ValueError("Embedding U contains NaN/inf.")
    labels0, _ = kmeans_own(U, k, n_init=12, max_iter=250, tol=1e-6, random_state=random_state)
    return labels0 + 1

In [6]:
def write_assignment_csv(path_out: str, labels1: np.ndarray):
    with open(path_out, "w", encoding="utf-8") as f:
        for i, lab in enumerate(labels1, start=1):
            f.write(f"{i}, {int(lab)}\n")

def run_on_file(path: str, out_dir: str, k_max_unc: int = 30):
    t0 = time.perf_counter()
    A = read_adjacency_csv(path)
    k = parse_k_from_filename(path)
    if k is None:
        L, _ = normalized_laplacian(A)
        k, _ = estimate_k_eigengap(L, k_max=k_max_unc)

    labels1 = spectral_clustering_own(A, k, random_state=RANDOM_SEED)
    dt = time.perf_counter() - t0

    out_csv = os.path.join(out_dir, os.path.basename(path))
    write_assignment_csv(out_csv, labels1)
    return dt, k, A.shape[0]

In [7]:
# ==============================
# RUN ALL BENCHMARK FILES
# ==============================

benchmark_files = list_benchmark_files(BENCHMARK_DIR)
benchmark_files

['Dataset\\D1-K=2.csv',
 'Dataset\\D1-UNC.csv',
 'Dataset\\D2-K=7.csv',
 'Dataset\\D2-UNC.csv',
 'Dataset\\D3-K=12.csv',
 'Dataset\\D3-UNC.csv']

In [8]:
out_dir = os.path.join(OUTPUT_ROOT, GROUP_NAME)
os.makedirs(out_dir, exist_ok=True)

results_rows = []
for path in benchmark_files:
    try:
        dt, k_used, n = run_on_file(path, out_dir=out_dir, k_max_unc=K_MAX_UNC)
        results_rows.append((os.path.basename(path), dt, k_used, n))
        print(f"OK   {os.path.basename(path)}   n={n}   K={k_used}   time={dt:.4f}s")
    except Exception as e:
        print(f"FAIL {os.path.basename(path)}   error={e}")

pd.DataFrame(results_rows, columns=["file_name", "execution_time_s", "k_used", "n_vertices"])

OK   D1-K=2.csv   n=34   K=2   time=0.0317s
OK   D1-UNC.csv   n=125   K=25   time=0.0803s
OK   D2-K=7.csv   n=62   K=7   time=0.0206s
OK   D2-UNC.csv   n=150   K=15   time=0.0681s
OK   D3-K=12.csv   n=115   K=12   time=0.0499s
OK   D3-UNC.csv   n=81   K=9   time=0.0414s


,file_name,execution_time_s,k_used,n_vertices
0,D1-K=2.csv,0.031662,2,34
1,D1-UNC.csv,0.080288,25,125
2,D2-K=7.csv,0.020641,7,62
3,D2-UNC.csv,0.068129,15,150
4,D3-K=12.csv,0.049921,12,115
5,D3-UNC.csv,0.041410,9,81


In [9]:
# Write description.txt (required)

desc_path = os.path.join(out_dir, "description.txt")
with open(desc_path, "w", encoding="utf-8") as f:
    f.write(f"{AUTHORS}\n")
    f.write(f"{REPO_URL}\n")
    for file_name, dt, k_used, n in results_rows:
        f.write(f"{{{file_name}, {dt:.6f}}}\n")

print("Wrote:", desc_path)
print("Outputs folder:", out_dir)

Wrote: .\Muhammad\description.txt
Outputs folder: .\Muhammad
